#### Model Monitor Table
##### Preprocess baseline feature table for model monitor creation

In [ ]:
import os 
import sys 
from pyspark.sql.functions import lit, col 
from pyspark.sql.types import DoubleType


path = os.path.abspath(os.path.join(os.getcwd(), "../../"))
sys.path.append(path)

#### Get Environment Variables

In [ ]:
# Get enviornment variables
dbutils.widgets.text("env", "dev")
dbutils.widgets.text("catalog_name", "devaiml")
dbutils.widgets.text("schema_name", "occupancy_project")
dbutils.widgets.text("source_table_name", "dev_occupancy_feature")
dbutils.widgets.text("monitor_table_name", "dev_occupancy_feature_monitor")

env = dbutils.widgets.get("env")
catalog_name = dbutils.widgets.get("catalog_name")
schema_name = dbutils.widgets.get("schema_name")
source_table_name = dbutils.widgets.get("source_table_name")
monitor_table_name = dbutils.widgets.get("monitor_table_name")

In [ ]:
# feature table
feature_table = f"{catalog_name}.{schema_name}.{source_table_name}"
monitor_table = f"{catalog_name}.{schema_name}.{monitor_table_name}"

# Read the feature table and add model_id column and cast Occupancy column to DoubleType
df = spark.table(feature_table) \
    .withColumn("model_id", lit(0)) \
    .withColumn("Occupancy", col("Occupancy").cast(DoubleType())) 

# Write the feature table to the monitor table
df \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .option("delta.enableChangeDataFeed", "true") \
    .saveAsTable(monitor_table)